In [3]:
#Importing Necessary Libraries
import cv2
import torch
import easyocr
import imutils
import numpy as np

#Reading the required image using OpenCV

test_image = cv2.imread(r"C:\Users\Nikhill\Downloads\SDIM\nissan3.jpg",cv2.IMREAD_COLOR) #change location as per the picture's path.
test_image = cv2.resize(test_image,(850,650))

#Converting the given image into grayscale for drawing contours & detecting edges.

gray_image = cv2.cvtColor(test_image,cv2.COLOR_BGR2GRAY) 
gray_image = cv2.bilateralFilter(gray_image,13,17,17)
#gray_image = cv2.medianBlur(gray_image,7) [[This filter can be used in case the Bilateral filter does not perform well]]

#detecting edges to reduce the area and predict a possible license plate

edges = cv2.Canny(gray_image,33,180)

#finding contours based on the detected edges 

cnts = cv2.findContours(edges.copy(),cv2.RETR_TREE,cv2.CHAIN_APPROX_TC89_L1)
cnts = imutils.grab_contours(cnts)
cnts = sorted(cnts,key=cv2.contourArea,reverse=True)[:12] #accepts the first 12 contours detected to prevent overfitting.

#contours found is approximated to a maximum of 4 points(edges), which typically represent a rectangle

cntsfound = None
for l in cnts:
    rect = cv2.arcLength(l,True)
    approx_points = cv2.approxPolyDP(l,0.020*rect,True)
 
    if len(approx_points) == 4:
        cntsfound = approx_points
        break
        
#Contours are drawn only if possible points of a rectangle are detected

if cntsfound is None:
    print ("Contours are absent")
else:
    cv2.drawContours(test_image,[cntsfound],0,(0,255,255),4)

#The blurred image surrounding the detected rectangle is not needed hence we will mask it to make it more accurate for the detected contours

mask = np.zeros(gray_image.shape,np.uint8)
rough_img = cv2.drawContours(mask,[cntsfound],0,255,0)
rough_img = cv2.bitwise_and(test_image,test_image,mask=mask)                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             

#Crop ROI (License Plate) from the main picture 

(l, w) = np.where(mask == 255)
(topl, topw) = (np.min(l), np.min(w))
(bottoml, bottomw) = (np.max(l), np.max(w))
lpimage = gray_image[topl:bottoml+1,topw:bottomw+1]

#using EasyOCR(Deep Learning PyTorch based framework to detect characters)

numberplate = easyocr.Reader(['en'],gpu=False)  #if GPU is present the second argument need not be passed
result = numberplate.readtext(lpimage,detail=0)
print("Detected Plate Number: ",result) 

#Output Sequence 
test_image = cv2.resize(test_image,(700,500))
lpimage = cv2.resize(lpimage,(500,300))
cv2.imshow("Detected License Plate",test_image)
cv2.imshow("Number Plate Image",lpimage)
cv2.waitKey(0)
cv2.destroyAllWindows()

Using CPU. Note: This module is much faster with a GPU.


Detected Plate Number:  ['SHI2 OMK']
